## Run this code in Databricks community edition, as data files are not available here, also see the video around this notebook below.

### https://www.youtube.com/watch?v=OednhGRp938&feature=youtu.be
#### Its a good example of building a model around raw text data
* [Full ML Workflow using Pipelines](https://docs.cloud.databricks.com/docs/latest/sample_applications/07%20Sample%20ML/MLPipeline%20Newsgroup%20Dataset.html)

In [ ]:
# Build, Inspect, and Tune ML Pipelines
### *In DSX*

A practical ML pipeline often involves a sequence of 
- data pre-processing, 
- feature extraction, 
- model fitting, 
- and validation stages.
For example, classifying text documents might involve text segmentation and cleaning, extracting features, and training a classification model with cross-validation.
Though there are many libraries we can use for each stage, connecting the dots is not as easy as it may look, especially with large-scale datasets.
Most ML libraries are not designed for distributed computation or they do not provide native support for pipeline creation and tuning.
Unfortunately, this problem is often ignored in academia, and it has received largely ad-hoc treatment in industry, where development tends to occur in manual one-off pipeline implementations.

In this notebook, we are going to use a simple text classification problem as an example to build an ML pipeline in Spark MLlib,
inspect it if it doesn't work as expected, and tune hyperparameters.

In [ ]:
# Let us first import some packages under `spark.ml`.
from pyspark.ml import *
from pyspark.ml.classification import *
from pyspark.ml.feature import *
from pyspark.ml.param import *
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import *

###Load "20 Newsgroups" dataset.

The dataset we are going to use is a simplified version of the popular 20 newsgroups dataset,
which is a collection of newsgroup articles labeled across 20 newsgroups.
The raw dataset can be obtained at https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups.
To simplify the demo, we preprocessed the dataset by mapping the 20 newsgroups into binary labels indicating whether it is related to science or not,
then we randomly split the dataset into training and test and save both in Parquet format, the default binary format used by Spark.

In [ ]:
%fs ls /databricks-datasets/news20.binary/data-001

In [ ]:
# Load the training dataset as a Spark DataFrame and cache it because we are going to access it multiple times.
training = sqlContext.read.parquet("/databricks-datasets/news20.binary/data-001/training").cache()

Call `display(...)` to show a DataFrame.
It also shows the columns, e.g.:
* topic: topic of the newsgroup,
* text: raw text of the newsgroup article,
* label: whether the topic is related to science (1) or not (0).

In [ ]:
display(training.limit(10))

We can explore this dataset more by checking the distribution of topics and visualizing the result.

In [ ]:
display(training.groupBy("topic").count())

By default, `display(...)` shows the result in a table.
It is simple to visualize the result with builtin charts.
For example, select the pie chart from the drop down list at the bottom of the table and set keys to "topic" and values to "count".
Then you should see the following pie (or donut) chart:

In [ ]:
display(training.groupBy("topic").count())

In this example notebook, we want to build an ML pipeline to predict the binary label, i.e., whether a newsgroup article is related to science or not.
So let us take a look at the label distribution.
This could be done by a simple `groupBy` method followed by `count`.
Similar to the pie chart above, you can visualize the result in a bar chart.
We can see the labels are not balanced but not very skewed either.

In [ ]:
display(training.groupBy("label").count())

Before we start building the pipeline, let us also load the test dataset for evaluation.

In [ ]:
test = sqlContext.read.parquet("/databricks-datasets/news20.binary/data-001/test").cache()

###Build an ML pipeline to classify newsgroup articles

As we mentioned in the introduction, a practical ML pipeline might consist of multiple stages like feature extraction, feature transformation, and model fitting.
We consider a very simple pipeline that consists of the following stages:

1. **RegexTokenizer**, which tokenizes each article into a sequence of words with a regex pattern,
2. **HashingTF**, which maps the word sequences produced by RegexTokenizer to sparse feature vectors using the hashing trick,
3. **LogisticRegression**, which fits the feature vectors and the labels from the training data to a logistic regression model.

<img src="http://spark.apache.org/docs/latest/img/ml-Pipeline.png" style="width: 800px;"/>

In [ ]:
# Constructing a pipeline is done by creating each pipeline stage and configuing its parameters.
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="s+")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features", numFeatures=5000)
lr = LogisticRegression(maxIter=20, regParam=0.01)

# To create an ML pipeline you concatenate a sequence of stages.
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [ ]:
# After we construct this ML pipeline, we can fit it using the training data
# and obtain a fitted pipeline model that can be used for prediction.
model = pipeline.fit(training)

### Check and evaluate prediction results

After we obtain a fitted pipeline model, we want to know how well it performs.
Let us start with some manual checks by displaying the predicted labels.

In [ ]:
# After fitting, making predictions is as simple as calling "transform" on the model.
prediction = model.transform(training)
# Show the predicted labels along with true labels and raw texts.
display(prediction.select("prediction", "label", "text").limit(10))

The predicted labels look accurate. However, we should evaluate the model quantitatively.

In [ ]:
# Create an evaluator for binary classification and use area under the ROC curve as the evaluation metric.
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
evaluator.evaluate(prediction)

The training metric is nearly perfect, but usually this is an indicator of overfitting.
So let's check test metric.

In [ ]:
# Call "model.transform" on test data and then evaluate the result.
evaluator.evaluate(model.transform(test))

The test metric is too low.
It seems something goes wrong rather than overfitting.
Let's inspect the pipeline stage by stage.

##Inspect a pipeline

Prediction from a pipeline actually contains all intermediate outputs from each stage:
* "words" is from the tokenzier,
* "features" is from the hashing TF transformer,
* "prediction", "probability", and "rawPredictions" are from the logistic regression model.

We can see this from the schema of "prediction".

In [ ]:
prediction.printSchema()

So let's check all of them by calling "display" on "prediction" directly.
For columns with complex types like "words", which contains arrays of strings, you can expand the result by clicking the small caret button on the top left.
We can see the tokenized words look quite weird.
There must be something wrong with the tokenizer.

In [ ]:
display(prediction.limit(10))

To check the tokenizer, we use `explainParams` to show the embedded documentation of its params.

In [ ]:
print tokenizer.explainParams()

It also shows the default value (if set) and the current value.
Oh, we forgot the backslash in the regex pattern for the tokernizer, which should be "\s+" but not "\s+".
Let's correct it.

In [ ]:
# Set the value of "pattern" back to "\s+".
tokenizer.setPattern("\\s+")

In [ ]:
# Fit the pipeline again.
model = pipeline.fit(training)

In [ ]:
# Check the predictions and make sure the tokenized words look good.
prediction = model.transform(training)
display(prediction.limit(10))

In [ ]:
# Check training metric.
evaluator.evaluate(prediction)

In [ ]:
evaluator.evaluate(model.transform(test))

It looks better now.
But the model is still overfitted.
As in all ML pipelines, we need to tune the hyperparameters to reduce the generalization error.

## Tune hyperparameters via cross validation

Cross validation (see [Wikipedia](https://en.wikipedia.org/wiki/Cross-validation_&#40;statistics&#41;)) is commonly used for hyperparameter tuning.
MLlib implements a version called [k-fold cross validation](https://en.wikipedia.org/wiki/Cross-validation_&#40;statistics&#41;#k-fold_cross-validation).
It takes a list of hyperparameter combinations and an evaluation metric, then searches for the best hyperparameter combination using cross validation.

In [ ]:
# We generate hyperparameter combinations by taking the cross product of some parameter values we want to try.
# For simplicity, we try different number of features in hashing TF transformer, and regularization parameters in logistic regression.
paramGrid = ParamGridBuilder() \
  .addGrid(hashingTF.numFeatures, [1000, 10000]) \
  .addGrid(lr.regParam, [0.05, 0.2]) \
  .build()

# Create a cross validator to tune the pipeline with the generated param grid.
cv = CrossValidator(estimator=pipeline,
                    evaluator=evaluator,
                    estimatorParamMaps=paramGrid,
                    numFolds=2)

Fitting a cross validation model is the same as fitting a pipeline.
It takes longer (depending on the number of hyperparameter combinations to try) to run due to cross validation.

In [ ]:
cvModel = cv.fit(training)

Let's check evaluation metrics again.

In [ ]:
evaluator.evaluate(cvModel.transform(training))

In [ ]:
evaluator.evaluate(cvModel.transform(test))

The test metric got improved.
For simplicity, we only tested a small number of hyperparameter combinations.
We can improve this further by trying more combinations, possibly with a bigger Spark cluster.

This is it, and thanks for reading this notebook!
You can find more details about ML pipelines in Spark in Joseph's Spark Summit talk embedded below
or the latest (http://spark.apache.org/docs/latest/ml-guide.html)[MLlib user guide].

In [ ]:
displayHTML("""https://www.youtube.com/embed/OednhGRp938""")